In [1]:
import json
import logging
import sys
from pathlib import Path

from sandbox import AgentVMConfig, AgentVMManager, VMServerClient

# Logger setup
logger = logging.getLogger("FastAPITest")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")
handler.setFormatter(formatter)
logger.handlers = [handler]


In [ ]:
TEST_CASES = [
    {
        "name": "Basic code execution",
        "method": "execute_code",
        "args": {"code": "print('Hello from FastAPI')"},
    },
    {
        "name": "Code with package install",
        "method": "execute_code",
        "args": {
            "code": "import requests\nprint('Requests version:', requests.__version__)",
            "packages": ["requests"],
        },
    },
    {
        "name": "GUI interaction with screenshot",
        "method": "execute_gui_code",
        "args": {
            "code": "import pyautogui\nimport time\npyautogui.click(x=35, y=60, button='left', clicks=2, interval=0.25)\ntime.sleep(5)\nprint('GUI interaction done')",
            "packages": ["pyautogui"],
        },
    },
    {
        "name": "Screenshot only",
        "method": "take_screenshot",
        "args": {},
    },
]


: 

In [ ]:
cfg = AgentVMConfig(
    container_name="test-agent-vm",
    host_server_dir=Path("sandbox/server"),
    host_sandbox_server_port=8765,
    host_sandbox_server_host="localhost",
    sandbox_server_port=8765,
    sandbox_server_host="0.0.0.0",
    sandbox_server_dir=Path("/home/user/server"),
)

vm = AgentVMManager(cfg, logger=logger)
vm.__enter__()  # Simulate entering the context manager

base_url = f"http://{cfg.host_sandbox_server_host}:{cfg.host_sandbox_server_port}"
client = VMServerClient(base_url)


def run_tests(client):
    logger.info("🚀 Running FastAPI sandbox tests...\n")
    for test in TEST_CASES:
        logger.info(f"▶️  {test['name']}")
        method = getattr(client, test["method"], None)
        if not method:
            logger.error(f"❌ Unknown method: {test['method']}")
            continue
        try:
            result = method(**test["args"])
            print(json.dumps(result, indent=2))
        except Exception as e:
            logger.error(f"❌ Error running test '{test['name']}': {e}")
        print()


run_tests(client)


In [ ]:
client.record_start()


# logger.info(json.dumps(vm.ssh.exec_command("ls"), indent=2))
# logger.info(json.dumps(client.execute_code(code="print('Manual code run')"), indent=2))


# code = """"
# import pyautogui
# import time

# pyautogui.click(x=35, y=60, button='left', clicks=2, interval=0.25)
# time.sleep(5)

# """

# # logger.info(json.dumps(client.execute_gui_code(code=)))

In [ ]:
vm.__exit__(None, None, None)
